<a href="https://colab.research.google.com/github/takky0330/GNN/blob/main/torch_geometric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

In [ ]:
#'''
# ----- 例: ユーザ + 設問を含む小さなグラフを仮定 -----
# X: ノード特徴量行列 (num_nodes x num_features)
# y: ノードラベル (num_nodes)
# edge_index: (2 x num_edges) の隣接情報 [source_nodes; target_nodes]

X = torch.tensor([
    [0.1, 0.5],  # User1の特徴量 (例:属性,すでに回答した質問数など)
    [0.2, 0.6],  # User2
    [0.9, 0.3],  # Question1
    [0.8, 0.4]   # Question2
], dtype=torch.float)

y = torch.tensor([1, 0, 2, 2])  # ノード分類用のラベル (ユーザノード: 0 or 1, 質問ノード: 2など 適当に仮定)
edge_index = torch.tensor([
    [0, 1, 0, 2, 1, 3],  # sourceノードID
    [1, 0, 2, 0, 3, 1]   # targetノードID
], dtype=torch.long)

data = Data(x=X, edge_index=edge_index, y=y)

In [ ]:
# 例: 8ノード (ユーザ/設問などの混在を想定)、3次元のノード特徴量
X = torch.tensor([
    [0.1, 0.5, 0.2],  # Node 0
    [0.2, 0.6, 0.3],  # Node 1
    [0.9, 0.3, 0.1],  # Node 2
    [0.8, 0.4, 0.4],  # Node 3
    [0.4, 0.8, 0.5],  # Node 4
    [0.2, 0.1, 0.7],  # Node 5
    [0.7, 0.9, 0.6],  # Node 6
    [0.5, 0.2, 0.8],  # Node 7
], dtype=torch.float)

# 例: 各ノードのクラス (0, 1, 2) のいずれかとする
y = torch.tensor([2, 0, 1, 2, 0, 1, 2, 0], dtype=torch.long)

# edge_index: (2 x E) のテンソルで無向エッジを定義
# 下記は一例として、ノードを適当に双方向で結んでいます
edge_index = torch.tensor([
    [0, 1, 0, 2, 1, 3, 2, 4, 3, 5, 4, 6, 5, 7, 6, 7, 6, 2],
    [1, 0, 2, 0, 3, 1, 4, 2, 5, 3, 6, 4, 7, 5, 7, 6, 2, 6]
], dtype=torch.long)

data = Data(x=X, edge_index=edge_index, y=y)

In [ ]:
#'''
# ----- GCNモデル定義 -----
class SimpleGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
class SimpleGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
#'''
model = SimpleGCN(in_channels=2, hidden_channels=4, out_channels=3)  # ラベルが3種類想定
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 入力次元(in_channels)は3、出力次元(out_channels)はここでは3クラス想定
model = SimpleGCN(in_channels=3, hidden_channels=4, out_channels=3)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# ----- 学習ループ -----
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# ----- 推論 (ノード分類の例) -----
model.eval()
pred = model(data).argmax(dim=1)
print("Predicted labels:", pred)